In [1]:
# BERT imports
from sklearn.model_selection import StratifiedShuffleSplit
from transformers import BertTokenizer
import pandas as pd
import numpy as np
import string
import time
import re

In [2]:
import nltk                                # Natural Language Toolkit
from nltk.corpus import stopwords          # module for stop words that come with NLTK
import tensorflow as tf
import tensorflow.keras.losses

In [3]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
from sklearn.metrics import roc_auc_score

In [4]:
# !pip install tensorflow_text

In [5]:
import tensorflow_hub as hub
import tensorflow_text as text



In [6]:
labels_all = pd.read_csv("data/labels_all.csv")
labels_all.drop(['Unnamed: 0'], axis=1, inplace=True)

text_all = pd.read_csv('data/text_all.csv')
text_all.drop(['Unnamed: 0'], axis=1, inplace=True)

#  Create binary labels for id
disease_types = ['Hypertriglyceridemia','Venous Insufficiency','Asthma','Gout','OSA','PVD','Gallstones','OA','GERD',
                'Depression','Obesity','CHF','Hypercholesterolemia','CAD','Diabetes','Hypertension']

In [7]:
# Make the outcome variables (disease types) binary
def make_binary(labels_df):
    for i in range(16):
        # In the disease column, code the disease of interest as 1 and other disease types as 0 to indicate 
        # whether this row is about this particular disease
        labels_df["disease_"+str(i)] = [1 if x==disease_types[i] else 0 for x in labels_df["disease"]]
        # In the label column, code Y as 1 and everything else as 0 to indicate whether a disease is present
        # (it may or may not be about this particular disease)
        labels_df["label_"+str(i)] = [1 if x=="Y" else 0 for x in labels_df["label"]] 
        # Create a "This_Disease" column that indicates whether a patient has this particular disease or not
        # A patient is coded as having this disease only when (disease is 1) AND (label is 1)
        labels_df["This_Disease_"+str(i)] = labels_df["disease_"+str(i)] * labels_df["label_"+str(i)] 
        labels_df.drop(["disease_"+str(i), "label_"+str(i)], axis=1, inplace=True)

    # Now that all useful info in "disease" and "Label" are combined in column "This_Disease", no longer need disease and label
    labels_df.drop(['label', 'disease'], axis=1, inplace=True)
    # Duplicates after removing disease and label are due to 0's in the "disease" column if the patient had other disease info
    labels_df.drop_duplicates(inplace = True)
    # Aggregate the binary values into one row for each id
    labels_df = labels_df.groupby('id').aggregate('sum').reset_index()
    
    return(labels_df)

In [8]:
binary_labels = make_binary(labels_all)

# Calculate the disease prevalence for each disease type
prevalences = (binary_labels.aggregate('sum')/1237).iloc[1:,]

# Combine the text data and the labels
df = text_all.merge(binary_labels, on="id", how="left")

del labels_all, text_all, binary_labels

In [9]:
# df = df.iloc[:50,]

In [10]:
# Download the stopwords from NLTK
nltk.download('stopwords')

# Import the standard English stop words list from NLTK
stopwords_english = stopwords.words('english') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
def preprocessing(text):
    text = "".join([x for x in text if x not in string.punctuation]) # Remmove punctuations 
    text = "".join([x.lower() for x in text]) # Convert to lower case
    text = ' '.join(['' if (x in stopwords_english) else x for x in text.split()]) # Remove stopwords
    text = re.sub("(\W|\d+|\n)", " ", text).strip() # remove spaces, digits and line breaks
    text = ' '.join(['' if (len(x) <= 2) else x for x in text.split()]) # Remove short words
    return(text)

In [12]:
df['text'] = [preprocessing(x) for x in df['text']]

In [13]:
# need to change it to stratified split later
# x_train,x_test ,y_train,y_test = train_test_split(df['text'], df['This_Disease_14'], random_state = 1, test_size = 0.25)

### Model:

In [14]:
# preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [15]:
bert_encoder = hub.KerasLayer(encoder_url, trainable=False)

INFO:absl:Using C:\Users\rakovski\AppData\Local\Temp\tfhub_modules to cache modules.


In [16]:
# bert_preprocess(["Machine Learning is cool!", "This model is amazing."])
# text_inputs = ["Machine Learning is cool!", "This model is amazing."]

In [17]:
preprocessor = hub.load(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [18]:
# # Step 1: tokenize batches of text inputs.
# text_inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')]
# tokenize = hub.KerasLayer(preprocessor.tokenize)
# tokenized_inputs = [tokenize(segment) for segment in text_inputs]

# # Step 3: pack input sequences for the Transformer encoder.
# seq_length = 512  
# bert_pack_inputs = hub.KerasLayer(
#     preprocessor.bert_pack_inputs,
#     arguments=dict(seq_length=max_length))  # Optional argument.
# encoder_inputs = bert_pack_inputs(tokenized_inputs)

In [19]:
# outputs = bert_encoder(encoder_inputs)
# # Neural network layers
# l = tf.keras.layers.Dropout(0.3, name="dropout")(outputs['pooled_output'])
# l = tf.keras.layers.Dense(128, activation='relu', name="dense")(l)
# l = tf.keras.layers.Dropout(0.3, name="dropout2")(l)
# y = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# # Use inputs and outputs to construct a final model
# model = tf.keras.Model(inputs=[text_inputs], outputs = [y])
# model.layers[3].trainable = False

In [20]:
# model.summary()

In [21]:
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

In [22]:
# Define a function to evaluate the model performance in terms of F1 score
def evaluate(model, X, y):
    pred = model.predict(X)
    pred = [1 if p>=0.5 else 0 for p in pred]
    acc = np.sum(y == pred)/len(y)
    
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()

    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = (2*precision*recall)/(precision + recall)
    
    y_pred = model.predict(X)
    auc_roc = round(roc_auc_score(y, y_pred),4)
    pre, rec, thresholds = precision_recall_curve(y, y_pred)
    auc_pr = round(auc(rec, pre),4)
    return(auc_roc, auc_pr, acc, precision, recall, specificity, f1)

In [ ]:
X = df['text']

epochs = 3
max_length = 512
batch_size = 32

time_1 = time.time()

for i in range(16): # Go through each disease type #####################################

    event_categorical = df['This_Disease_'+str(i)]
    dis_type = disease_types[i]
    
    time_start = time.time()   

    # Split the data into train and test sets with stratification
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.25,random_state=0)
    
    j = 0
    for train_index, test_index in sss.split(X, event_categorical):   
              
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = event_categorical[train_index], event_categorical[test_index]
        
        j += 1
        iteration = "iter" + str(j)

        # tokenize batches of text inputs.
        text_inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')]
        tokenize = hub.KerasLayer(preprocessor.tokenize)
        tokenized_inputs = [tokenize(segment) for segment in text_inputs]

        # pack input sequences for the Transformer encoder.       
        bert_pack_inputs = hub.KerasLayer(
            preprocessor.bert_pack_inputs,
            arguments=dict(seq_length=max_length))  # Optional argument.
        encoder_inputs = bert_pack_inputs(tokenized_inputs)
        
        # BERT model:
        outputs = bert_encoder(encoder_inputs)

        x = tf.keras.layers.Dropout(0.3, name="dropout")(outputs['pooled_output'])
        x = tf.keras.layers.Dense(128, activation='relu', name="dense")(x)
        x = tf.keras.layers.Dropout(0.3, name="dropout2")(x)
        y = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(x)

        # Use inputs and outputs to construct a final model
        mymodel = tf.keras.Model(inputs=[text_inputs], outputs = [y])
        mymodel.layers[3].trainable = False
        mymodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

        time_s = time.time() 
        mymodel.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
        time_e = time.time() - time_s

        # Collect and log evaluation metrics
#             auc_roc, auc_pr, acc, precision, recall, specificity, f1 = evaluate(model, x_test, y_test[:,1])
        auc_roc, auc_pr, acc, precision, recall, specificity, f1 = evaluate(mymodel, x_test, y_test)

        with open('other/CNN_Paper_no_embedding_BERT2.csv','a') as fd:
            fd.write(f'{dis_type},{iteration},{auc_roc},{auc_pr},{acc},{precision},{recall},{specificity},{f1},{time_e}\n')
        
        del x_train, x_test,text_inputs,tokenize,tokenized_inputs,bert_pack_inputs,encoder_inputs,outputs,x,y,mymodel
    
    # Average metrics
    running_time = time.time() - time_start
#     logging.info(f',{disease_types[i]}, running time, {running_time}')
    with open('other/CNN_Paper_no_embedding_BERT2.csv','a') as fd:
        fd.write(f'{dis_type}, 10 iteration running time, {running_time}\n')

Epoch 1/3
29/29 [==============================] - 481s 16s/step - loss: 0.2673 - auc: 0.4641
Epoch 2/3
29/29 [==============================] - 467s 16s/step - loss: 0.2406 - auc: 0.4310
Epoch 3/3
29/29 [==============================] - 467s 16s/step - loss: 0.2160 - auc: 0.5278


<ipython-input-22-03a631c9fcc7>:9: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = tp/(tp + fp)


Epoch 1/3
29/29 [==============================] - 500s 17s/step - loss: 0.2828 - auc: 0.5022
Epoch 2/3
29/29 [==============================] - 506s 17s/step - loss: 0.2235 - auc: 0.4595
Epoch 3/3
29/29 [==============================] - 516s 18s/step - loss: 0.2299 - auc: 0.4764


<ipython-input-22-03a631c9fcc7>:9: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = tp/(tp + fp)


Epoch 1/3
29/29 [==============================] - 478s 16s/step - loss: 0.2606 - auc: 0.4654
Epoch 2/3
29/29 [==============================] - 492s 17s/step - loss: 0.2297 - auc: 0.4572
Epoch 3/3
29/29 [==============================] - 494s 17s/step - loss: 0.2265 - auc: 0.5292


...

Epoch 1/3
29/29 [==============================] - 511s 18s/step - loss: 0.2860 - auc: 0.5507
Epoch 2/3
29/29 [==============================] - 570s 20s/step - loss: 0.2269 - auc: 0.4738
Epoch 3/3
29/29 [==============================] - 640s 22s/step - loss: 0.2238 - auc: 0.4829


<ipython-input-22-03a631c9fcc7>:9: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = tp/(tp + fp)


Epoch 1/3
29/29 [==============================] - 573s 20s/step - loss: 0.3172 - auc: 0.4953
Epoch 2/3
29/29 [==============================] - 605s 21s/step - loss: 0.2170 - auc: 0.5640
Epoch 3/3
29/29 [==============================] - 655s 23s/step - loss: 0.2137 - auc: 0.5444


<ipython-input-22-03a631c9fcc7>:9: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = tp/(tp + fp)


Epoch 1/3
28/29 [===========================>..] - ETA: 20s - loss: 0.2567 - auc: 0.4992

In [ ]:
time_2 = time.time() - time_1  
with open('other/CNN_Paper_no_embedding_BERT2.csv','a') as fd:
    fd.write(f'{dis_type}, Total running time, {time_2}\n')

In [ ]:
# from sklearn import metrics
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import average_precision_score, precision_recall_curve
# from sklearn.metrics import auc, plot_precision_recall_curve
# from sklearn.metrics import roc_auc_score

# evaluate(model, x_test, y_test)

In [ ]:
# Diabetes: acc=0.6923076923076923, bert tokenizatin time: , training time: 2177.546822309494 with 0.5 dropout rate
# Diabetes: acc=, bert tokenizatin time: , training time: 1586.164499759674  with 0.3 dropout rate, untrainable
# Diabetes: acc=0.5384615384615384, training time: 1255, dense layer (128), trainable, lr=0.00001.#error
# Diabetes: acc=0.46153846153846156, training time: 1447, dense layer (128),  trainable, lr=default
# Diabetes: acc=0.46153846153846156, training time:420, no dense layer,  untrainable, lr=default.
# Diabetes: CNN: acc=0.8258064516129032
# Diabetes: untrainable, dense 128.
# (0.5532,
#  0.7029,
#  0.667741935483871,
#  0.667741935483871,
#  1.0,
#  0.0,
#  0.8007736943907157)
# Diabetes: trainable, dense 128. Training time: 29084.541052103043
# (0.4852,
#  0.653,
#  0.667741935483871,
#  0.667741935483871,
#  1.0,
#  0.0,
#  0.8007736943907157)